In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import consolidateFiles as cf
import datacleaning as cl
import fragmentation as fr
import hervpd as hp

from datetime import datetime

## ! Parse activity files and parse interval files are to be replaced with the corresponding database queries as soon as they are available 

## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [3]:
verbose = True

PATH = "C:\\Users\\ju\\GDrive\\Projects\\HeRV\\Data\\"
RAW_PATH = PATH + "Raw"
PRE_PATH = PATH + "PreProcessed"

sessfile = PRE_PATH + "\\sessions.xlsx"  


## 1 - Read sessions

---

In [4]:
df = pd.read_excel(sessfile)
df.sample(1)

,activity,beatscount,duration,hf,hfnu,lf,lf_hf,lfnu,mhr,mrri,...,posture,removed_artifacts,rmssd,sdnn,start,stop,total_power,user,vlf,sess_id
102,commute,1382,790,88.312919,15.894033,467.322775,5.291669,84.105967,102.856543,585.63314,...,sit,0,15.856548,37.393547,2017-11-01 10:25:52,2017-11-01 10:39:02,812.498104,0,256.86241,102


In [ ]:
sessions = df.to_dict(orient='records')
%time sessions = cf.sessions_add_beats(sessions, RAW_PATH, verbose=False)

## 2 - Generate fragments from sessions

Breaks the sessions duration in fragments

Configurations:
* duration of each fragment in seconds;
* number of seconds to be discarded at the beginning of the session, accounting for user's stabilization and adjustment to posture and activity 

---


In [ ]:
# duration (in seconds) to be cropped from the beginning of each second to account for stabilization and user adjustment
crop = 0
    
# duration (in seconds) of each fragment to be sent to analysis
duration = 60
    
# if any fragment has more than 'threshold' consecutive seconds with no beats, it will be discarded
threshold = 3    

%time frags = fr.gen_fragments_dataset(sessions, duration, crop)

In [ ]:
# [(s['activity'], s['start'], s['stop']) for s in sessions if s['start'] > datetime(2017,10,14,11) and s['stop'] < datetime(2017,10,14,23) and s['user'] == 0]

## 3 - Save

---

In [ ]:
df = pd.DataFrame(frags)
df.describe()

In [ ]:
df[(df.user == 2 ) & (df.start > datetime(2017,10,26,20,40)) & (df.start < datetime(2017,10,26,20,59))]

In [ ]:
filename = PRE_PATH + '\\df_' + str(duration) + '_' + str(crop) + '.xlsx'
print(filename)
df.to_excel(filename)

# Applying all steps above to generate different datasets

In [ ]:
# durations = [300, 240, 180, 150, 120, 90, 60]
# crops = [120, 90, 60]

# durations = [20, 30, 60, 90, 120, 150, 180]
# crops = [10, 30]

durations = [30, 300]
crops= [30]

def multifrag(sessions, durations, crops, path_out):
    for cr in crops:
        for dr in durations:            
            print ('generating dataset for duration', dr, 'and crop', cr, '...')
            ds = fr.gen_fragments_dataset(sessions, dr, cr)
            print('resulting dataset:', len(ds), 'records' )
            fname = path_out + '\\df_' + str(dr) + '_' + str(cr) + '.xlsx'            
            ds.to_excel(fname)

In [ ]:
%time multifrag(sessions, durations, crops, PRE_PATH)

## ( Extra - save files for LDA Grover)

---

In [ ]:
dfr[['activity']].to_csv('./classifications')

In [ ]:
def get_ints(beats):
    return [beat['interval'] for beat in beats]

dfr['ts'] = dfr['rr'].apply(get_ints)


In [ ]:
dfr.loc[['ts']].sample(3)

In [ ]:
dfr[['ts']].to_csv('./timeseries')

In [ ]:
dfr[cl.features_all].to_csv('./features')

In [ ]:
dfr.describe()

In [ ]:
for a in d.activity.unique()
       df.groupby(column).count()['user']